In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 将彩色图像转换为灰度图像
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 随机划分训练集和验证集
train_dataset, val_dataset = random_split(dataset, [40000, 10000])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [2]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32 * 32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        logits = self.linear_relu_stack(x)
        return logits

model = SimpleMLP()

In [3]:
def train(model, loader, criterion, optimizer):
    model.train()
    for data, target in loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return correct / total

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer)
    accuracy = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {accuracy*100:.2f}%')

Epoch 1/10, Validation Accuracy: 35.98%
Epoch 2/10, Validation Accuracy: 41.51%
Epoch 3/10, Validation Accuracy: 42.62%
Epoch 4/10, Validation Accuracy: 42.63%
Epoch 5/10, Validation Accuracy: 45.40%
Epoch 6/10, Validation Accuracy: 43.81%
Epoch 7/10, Validation Accuracy: 45.61%
Epoch 8/10, Validation Accuracy: 44.45%
Epoch 9/10, Validation Accuracy: 44.02%
Epoch 10/10, Validation Accuracy: 44.19%
